In [37]:
"""
Обработка результатов опросника Сфера профессиональных предпочтений учащихся. (модификация Г. В. Резапкиной)
"""

'\nОбработка результатов опросника Сфера профессиональных предпочтений учащихся. (модификация Г. В. Резапкиной)\n'

In [38]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time
import re

In [39]:
file_data_xlsx_sppu = 'data/SPPU.xlsx'
path_to_end_folder_sppu = 'data'
threshold_sppu =8

In [40]:
def processing_result_sppu(row):
    """
    Обработка результатов тестирования
    """

    # Создаем словарь для хранения данных
    dct_type = {'сфера работы с людьми': 0, 'сфера умственного труда': 0, 'сфера технических интересов': 0,
                'сфера эстетики и искусства': 0,
                'сфера физического труда, подвижной деятельности': 0,
                'сфера материальных интересов, планово-экономических видов работ': 0}
    dct_error = {}  # словарь для хранения ошибочных  значений, для того чтобы было легче находить ошибки при обновлении

    # 1
    if row[0] == 'общаться с самыми разными людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[0] == 'что-нибудь делать своими руками – мебель, одежду, машины и т.д.;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[0] == 'снимать фильмы, рисовать, писать книги, выступать на сцене и т.д.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №1'] = f'Полученное значение-{row[0]} не совпадает с эталонными:[общаться с самыми разными людьми;], [что-нибудь делать своими руками – мебель, одежду, машины и т.д.;], [снимать фильмы, рисовать, писать книги, выступать на сцене и т.д.]'

    # 2
    if row[1] == 'художественная форма, мастерство писателя или режиссера;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[1] == 'сюжет, действие героев;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[1] == 'информация, которая может пригодиться в жизни.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №2'] = f'Полученное значение-{row[1]} не совпадает с эталонными:[художественная форма, мастерство писателя или режиссера;], [сюжет, действие героев;], [информация, которая может пригодиться в жизни.]'

    # 3
    if row[2] == 'в области науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[2] == 'за общественную деятельность;':
        dct_type['сфера работы с людьми'] += 1
    elif row[2] == 'в области искусства.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №3'] = f'Полученное значение-{row[2]} не совпадает с эталонными:[в области науки;], [за общественную деятельность;], [в области искусства.]'

    # 4
    if row[3] == 'управляющим банка;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[3] == 'главным инженером на производстве;':
        dct_type['сфера технических интересов'] += 1
    elif row[3] == 'начальником экспедиции.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №4'] = f'Полученное значение-{row[3]} не совпадает с эталонными:[управляющим банка;], [главным инженером на производстве;], [начальником экспедиции.]'

    # 5
    if row[4] == 'достижения науки;':
        dct_type['сфера умственного труда'] += 1
    elif row[4] == 'развитие производства;':
        dct_type['сфера технических интересов'] += 1
    elif row[4] == 'взаимопонимание среди людей.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №5'] = f'Полученное значение-{row[4]} не совпадает с эталонными:[достижения науки;], [развитие производства;], [взаимопонимание среди людей.]'

    # 6
    if row[5] == 'благоустройством школы (столовая, спортзал, компьютеры);':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[5] == 'созданием дружного, сплоченного коллектива;':
        dct_type['сфера работы с людьми'] += 1
    elif row[5] == 'разработкой новых технологий обучения.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №6'] = f'Полученное значение-{row[5]} не совпадает с эталонными:[благоустройством школы (столовая, спортзал, компьютеры);], [созданием дружного, сплоченного коллектива;], [разработкой новых технологий обучения.]'

    # 7
    if row[6] == 'внешний вид экспонатов (цвет, форма);':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[6] == 'внутреннее устройство экспонатов (механизм);':
        dct_type['сфера умственного труда'] += 1
    elif row[6] == 'практическое применение экспонатов.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №7'] = f'Полученное значение-{row[6]} не совпадает с эталонными:[внешний вид экспонатов (цвет, форма);], [внутреннее устройство экспонатов (механизм);], [практическое применение экспонатов.]'

    # 8
    if row[7] == 'мужество, смелость, выносливость;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[7] == 'дружелюбие, чуткость, отзывчивость;':
        dct_type['сфера работы с людьми'] += 1
    elif row[7] == 'ответственность, аккуратность.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №8'] = f'Полученное значение-{row[7]} не совпадает с эталонными:[мужество, смелость, выносливость;], [дружелюбие, чуткость, отзывчивость;], [ответственность, аккуратность.]'

    # 9
    if row[8] == 'писать стихи или музыку или рисовать;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[8] == 'ставить различные опыты;':
        dct_type['сфера умственного труда'] += 1
    elif row[8] == 'тренироваться.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №9'] = f'Полученное значение-{row[8]} не совпадает с эталонными:[писать стихи или музыку или рисовать;], [ставить различные опыты;], [тренироваться.]'

    # 10
    if row[9] == 'экстремальный туризм (альпинизм, виндсерфинг, горные лыжи);':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[9] == 'деловое общение;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[9] == 'возможность знакомства с историей и культурой другой страны.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №10'] = f'Полученное значение-{row[9]} не совпадает с эталонными:[экстремальный туризм (альпинизм, виндсерфинг, горные лыжи);], [деловое общение;], [возможность знакомства с историей и культурой другой страны.]'

    # 11
    if row[10] == 'о машине нового типа;':
        dct_type['сфера технических интересов'] += 1
    elif row[10] == 'о новой научной теории;':
        dct_type['сфера умственного труда'] += 1
    elif row[10] == 'о человеческих взаимоотношениях.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №11'] = f'Полученное значение-{row[10]} не совпадает с эталонными:[о машине нового типа;], [о новой научной теории;], [о человеческих взаимоотношениях.]'

    # 12
    if row[11] == 'технический;':
        dct_type['сфера технических интересов'] += 1
    elif row[11] == 'музыкальный;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[11] == 'спортивный.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №12'] = f'Полученное значение-{row[11]} не совпадает с эталонными:[технический;], [музыкальный;], [спортивный.]'

    # 13
    if row[12] == 'улучшению взаимопонимания между учителями и учениками;':
        dct_type['сфера работы с людьми'] += 1
    elif row[12] == 'поддержанию здоровья учащихся, занятиям спортом;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[12] == 'укреплению дисциплины.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №13'] = f'Полученное значение-{row[12]} не совпадает с эталонными:[улучшению взаимопонимания между учителями и учениками;], [поддержанию здоровья учащихся, занятиям спортом;], [укреплению дисциплины.]'

    # 14
    if row[13] == 'научно-популярные фильмы;':
        dct_type['сфера умственного труда'] += 1
    elif row[13] == 'программы о культуре и спорте;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[13] == 'спортивные программы.':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    else:
        dct_error['Вопрос №14'] = f'Полученное значение-{row[13]} не совпадает с эталонными:[научно-популярные фильмы;], [программы о культуре и спорте;], [спортивные программы.]'

    # 15
    if row[14] == 'с машинами, механизмами;':
        dct_type['сфера технических интересов'] += 1
    elif row[14] == 'с объектами природы;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[14] == 'с детьми или сверстниками.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №15'] = f'Полученное значение-{row[14]} не совпадает с эталонными:[с машинами, механизмами;], [с объектами природы;], [с детьми или сверстниками.]'

    # 16
    if row[15] == 'давать знания и умения;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[15] == 'учить общению с другими людьми;':
        dct_type['сфера работы с людьми'] += 1
    elif row[15] == 'обучать навыкам работы.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №16'] = f'Полученное значение-{row[15]} не совпадает с эталонными:[давать знания и умения;], [учить общению с другими людьми;], [обучать навыкам работы.]'

    # 17
    if row[16] == 'вести здоровый образ жизни;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[16] == 'иметь возможность заниматься творчеством;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[16] == 'иметь удобные бытовые условия.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №17'] = f'Полученное значение-{row[16]} не совпадает с эталонными:[вести здоровый образ жизни;], [иметь возможность заниматься творчеством;], [иметь удобные бытовые условия.]'

    # 18
    if row[17] == 'защита интересов и прав граждан;':
        dct_type['сфера работы с людьми'] += 1
    elif row[17] == 'забота о материальном благополучии людей;':
        dct_type['сфера технических интересов'] += 1
    elif row[17] == 'наука и технический прогресс.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №18'] = f'Полученное значение-{row[17]} не совпадает с эталонными:[защита интересов и прав граждан;], [забота о материальном благополучии людей;], [наука и технический прогресс.]'

    # 19
    if row[18] == 'физкультуры;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[18] == 'математики;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[18] == 'труда.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №19'] = f'Полученное значение-{row[18]} не совпадает с эталонными:[физкультуры;], [математики;], [труда.]'

    # 20
    if row[19] == 'планировать производство продукции;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[19] == 'изготавливать изделия;':
        dct_type['сфера технических интересов'] += 1
    elif row[19] == 'заниматься сбытом продукции.':
        dct_type['сфера работы с людьми'] += 1
    else:
        dct_error['Вопрос №20'] = f'Полученное значение-{row[19]} не совпадает с эталонными:[планировать производство продукции;], [изготавливать изделия;], [заниматься сбытом продукции.]'

    # 21
    if row[20] == 'о выдающихся ученых и их открытиях;':
        dct_type['сфера умственного труда'] += 1
    elif row[20] == 'о творчестве ученых и музыкантов;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[20] == 'об интересных изобретениях.':
        dct_type['сфера технических интересов'] += 1
    else:
        dct_error['Вопрос №21'] = f'Полученное значение-{row[20]} не совпадает с эталонными:[о выдающихся ученых и их открытиях;], [о творчестве ученых и музыкантов;], [об интересных изобретениях.]'

    # 22
    if row[21] == 'делая что-то по хозяйству;':
        dct_type['сфера технических интересов'] += 1
    elif row[21] == 'с книгой;':
        dct_type['сфера умственного труда'] += 1
    elif row[21] == 'на выставках, концертах и пр.':
        dct_type['сфера эстетики и искусства'] += 1
    else:
        dct_error['Вопрос №22'] = f'Полученное значение-{row[21]} не совпадает с эталонными:[делая что-то по хозяйству;], [с книгой;], [на выставках, концертах и пр.]'

    # 23
    if row[22] == 'о художественной выставке;':
        dct_type['сфера эстетики и искусства'] += 1
    elif row[22] == 'о ситуации на фондовой бирже;':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    elif row[22] == 'о научном открытии.':
        dct_type['сфера умственного труда'] += 1
    else:
        dct_error['Вопрос №23'] = f'Полученное значение-{row[22]} не совпадает с эталонными:[о художественной выставке;], [о ситуации на фондовой бирже;], [о научном открытии.]'

    # 24
    if row[23] == 'в помещении, где много людей;':
        dct_type['сфера работы с людьми'] += 1
    elif row[23] == 'в необычных условиях;':
        dct_type['сфера физического труда, подвижной деятельности'] += 1
    elif row[23] == 'в обычном кабинете.':
        dct_type['сфера материальных интересов, планово-экономических видов работ'] += 1
    else:
        dct_error['Вопрос №24'] = f'Полученное значение-{row[23]} не совпадает с эталонными:[в помещении, где много людей;], [в необычных условиях;], [в обычном кабинете.]'

    # проверяем на ошибки
    if len(dct_error) > 0:
        begin_str = 'Скопируйте правильные значения для указанных вопросов из квадратных скобок в вашу форму. \n'
        # создаем строку с результатами
        for sphere, value in dct_error.items():
            begin_str += f'{sphere} - {value};\n'
        return begin_str
    else:
        # сортируем по убыванию
        result_lst = sorted(dct_type.items() , key=lambda t : t[1] , reverse=True)
        begin_str = ''
        # создаем строку с результатами
        for sphere,value in result_lst:
            begin_str += f'{sphere} - {value};\n'


        #добавляем описание
        return begin_str


In [41]:
df = pd.read_excel(file_data_xlsx_sppu)


In [42]:
base_df = df.iloc[:,:threshold_sppu] # создаем датафрейм с данными не относящимися к тесту

In [43]:
# делаем строковыми названия колонок
base_df.columns = list(map(str,base_df.columns))
# заменяем пробелы на нижнее подчеркивание и очищаем от пробельных символов в начале и конце
base_df.columns = [column.strip().replace(' ','_') for column in base_df.columns]

In [44]:
# очищаем от всех символов кроме букв цифр
base_df.columns = [re.sub(r'[^_\d\w]','',column) for column in base_df.columns]

In [45]:
answers_df = df.iloc[:,threshold_sppu:] # датафрейм с результатами

In [46]:
answers_df.columns = [f'СППУ_Вопрос_ №_{i}' for i in range(1,answers_df.shape[1]+1)]

In [47]:
answers_df = answers_df.astype(str)

In [48]:
answers_df = answers_df.applymap(lambda x:x.strip())

In [49]:
answers_df['СППУ_Обработанный_результат'] = answers_df.apply(processing_result_sppu, axis=1)

answers_df['СППУ_Обработанный_результат'] = answers_df['СППУ_Обработанный_результат'].astype(str)

In [50]:
# соединяем после обработки
df = pd.concat([base_df,answers_df],axis=1)


In [51]:
# Создаем строку с описанием
description_result = """
Шкала оценки результатов
10-12 баллов – ярко выраженная профессиональная склонность;
7-9 баллов – средне выраженная профессиональная склонность;
4-6 баллов – слабо выраженная профессиональная склонность;
0-3 баллов – профессиональная склонность не выражена.

Примеры профессий для каждой сферы
Сфера работы с людьми: учитель, педагог, экскурсовод, воспитатель,  социолог, психолог, менеджер по персоналу, следователь.
Сфера умственного труда: ученый-исследователь (математик, физик, химик, кибернетик, археолог, геолог),  инженер, юрист, врач, эколог, архитектор, продюсер.
Сфера технических интересов:  программист, электротехник, радиотехник, Web-мастер, статистик, водитель, технолог, диспетчер, секретарь-машинистка, телефонист.
Сфера эстетики и искусства: художник, дизайнер, писатель, поэт,  режиссер, артист, конструктор, косметолог, костюмер, гример, кондитер, портной-кутюрье, цветовод. 
Сфера физического труда,  подвижной деятельности: спортсмен, фотограф, экспедитор, парикмахер,  бармен, официант, стюардесса, продавец, закройщик, специалист по ремонту, кассир, медперсонал, бригадир, кладовщик,  почтальон,  фермер,  водитель-дальнобойщик, полицейский, военный.
Сфера материальных интересов,  планово-экономических видов работ: экономист, администратор, менеджер, предприниматель, аудитор, специалист по рекламе, брокер, агент страховых компаний, коммерсант, завхоз.
"""

In [52]:
if 'ФИО' in df.columns:
    df['СППУ_Описание_результата'] = df['ФИО'] +'.'+ ' \nСфера профессиональных предпочтений учащихся.\n' + df['СППУ_Обработанный_результат'] + description_result
else:
    df['СППУ_Описание_результата'] ='Сфера профессиональных предпочтений учащихся.\n' + df['СППУ_Обработанный_результат'] + description_result

In [53]:
# генерируем текущее время
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

df.to_excel(f'{path_to_end_folder_sppu}/Полная таблица с результатами СППУ от {current_time}.xlsx', index=False,engine='xlsxwriter')

In [54]:

# Создаем сокращенный вариант
send_df = df.iloc[:,:threshold_sppu]
# Добавляем колонки с результатами
send_df['СППУ_Обработанный_результат'] = df['СППУ_Обработанный_результат']
send_df['СППУ_Описание_результата'] = df['СППУ_Описание_результата']
send_df.to_excel(f'{path_to_end_folder_sppu}/Краткая таблица с результатами СППУ  от {current_time}.xlsx',index=False,engine='xlsxwriter')